<div style="display: flex; background-color: RGB(255,114,0);" >
<h1 style="margin: auto; padding: 25px; ">Programme de détection des faux billets</h1>
</div>

## **Mise en production du modèle avec de nouvelles données**

In [3]:
#Importation des librairies joblib et pandas
import joblib
import pandas as pd

In [4]:
# Chargement du fichier csv contenant les nouvelles données
X_new = pd.read_csv('billets_production.csv')

In [5]:
X_new

,diagonal,height_left,height_right,margin_low,margin_up,length,id
0,171.76,104.01,103.54,5.21,3.30,111.42,A_1
1,171.87,104.17,104.13,6.00,3.31,112.09,A_2
2,172.00,104.58,104.29,4.99,3.39,111.57,A_3
3,172.49,104.55,104.34,4.44,3.03,113.20,A_4
4,171.65,103.63,103.56,3.77,3.16,113.33,A_5


In [6]:
# Nettoyage des colonnes inutiles
X_new = X_new.drop(columns=['id', 'diagonal', 'height_left'], errors='ignore')

# Chargement du pipeline
pipeline = joblib.load('pipeline_rlog_full.joblib')


In [7]:
# Application du seuil de décision améliorant le modèle puis réalisation de la prédiction
seuil = 0.740

# Prédiction des probabilités sur la classe positive
probas = pipeline.predict_proba(X_new)[:, 1]

# Application du seuil pour classification binaire
y_pred = (probas >= seuil).astype(int)

print(y_pred)

[0 0 0 1 1]


In [8]:
ids = pd.read_csv('billets_production.csv')['id']
# Construire le DataFrame avec id, probas et prédictions
resultats = pd.DataFrame({
    'id': ids,
    'Proba vrai billet': probas,
    'Prediction': y_pred
})
resultats

,id,Proba vrai billet,Prediction
0,A_1,0.000139,0
1,A_2,0.000018,0
2,A_3,0.000059,0
3,A_4,0.995039,1
4,A_5,0.999974,1


## **Résultats**

In [10]:
# Affichage des classes
print(pipeline.named_steps['model_rlog'].classes_)

[False  True]


In [11]:
# Comptabilisation du nombre de vrai et faux billets
nombre_faux = (resultats['Prediction'] == 0).sum()
nombre_vrais = (resultats['Prediction'] == 1).sum()

print(f"Nombre de faux billets prédits : {nombre_faux}")
print(f"Nombre de vrais billets prédits : {nombre_vrais}")

Nombre de faux billets prédits : 3
Nombre de vrais billets prédits : 2


### **Identification des billets selon la catégorie prédite(Vrai ou Faux)**

In [13]:
# Filtrage des id des vrais billets (Prediction == 1)
ids_vrais = resultats.loc[resultats['Prediction'] == 1, 'id'].tolist()

# Filtrage  des id des faux billets (Prediction == 0)
ids_faux = resultats.loc[resultats['Prediction'] == 0, 'id'].tolist()

print(f"Les id des vrais billets prédits sont : {ids_vrais}")
print(f"Les id des faux billets prédits sont : {ids_faux}")

Les id des vrais billets prédits sont : ['A_4', 'A_5']
Les id des faux billets prédits sont : ['A_1', 'A_2', 'A_3']
